# Configured Spark, PostgreSQL

##### 1. PySpark Imports

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import psycopg2

##### 2. Configuring SparkSession for PostgreSQl

<font color="yellow">Added the required jar for postgreSQL in the specified path <font color="aqua">check documentation

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/Users/deependrashekhawat/jars/postgresql-42.2.21.jar") \
    .getOrCreate()

##### 3. Database connection currsor

<font color="yellow">Added the library for postgreSQL <font color="aqua">check references

In [3]:
conn = psycopg2.connect(host="localhost", database="testrestaurant", user="postgres", password="Welcome@1", port=5436)
curr = conn.cursor()

# Load Cities and State

In [5]:
def insert_cities(data_file):
    df_cities = spark.read.csv(data_file, header=True)
    
    df_cities.select(F.trim(F.initcap("cities")), "state").registerTempTable("cities")
    output = spark.sql('select * from cities')
    
    city = [tuple(x) for x in output.collect()]
    joinValues = ','.join(['%s'] * len(city))
    insertValue = "INSERT INTO address (city, state) VALUES{}".format(joinValues)
    
    curr.execute(insertValue, city)
    
    conn.commit()

In [7]:
insert_cities("/Users/deependrashekhawat/Documents/BU/Semesters/Sem3/Project/PostgreSQL_DB/GA.csv")

# Load Cuisine

In [55]:
def insert_cuisine(data_file):
    df_cuisine = spark.read.csv(data_file, header=True)
    
    df_cuisine.select(F.trim(F.initcap("cuisine"))).registerTempTable("cuisine")
    output = spark.sql("select * from cuisine")
    
    cuisine = [tuple(x) for x in output.collect()]
    joinValues = ','.join(['%s'] * len(cuisine))
    insertValue = "INSERT INTO cuisines (cuisine_name) VALUES{}".format(joinValues)
    
    curr.execute(insertValue, cuisine)
#     curr.execute("select * from cuisines")   --Testing Purpose
#     result = curr.fetchmany(2)
#     for row in result:
#         print(row)
    
    conn.commit()

In [56]:
insert_cuisine("/Users/deependrashekhawat/Documents/BU/Semesters/Sem3/Project/PostgreSQL_DB/cuisine.csv")

# Load Data in restaurant staging table

##### 1. Reading Cuisine data from Database

In [4]:
curr.execute("SELECT cuisine_name from cuisines")
resultCuisine = [item for x in curr.fetchall() for item in x]

##### 2. Reading Restaurant CSV Data

In [5]:
restaurantJSON = "/Users/deependrashekhawat/Documents/BU/Semesters/Sem3/Project/PostgreSQL_DB/YelpDataset/yelp_academic_dataset_business.json"
business_data_raw = spark \
                .read \
                .json(restaurantJSON)

##### 3. Data PreProcessing and Loading to Staging

In [6]:
df_business_data_raw = business_data_raw \
    .select("name", "address", "city", "state", "postal_code", \
             F.explode(F.split(F.initcap("categories"), ",|\s+")).alias("cuisine"), \
             "latitude", "longitude", "stars", "review_count")

In [7]:
df_business_data = df_business_data_raw \
    .filter(df_business_data_raw.cuisine.rlike("|".join([".*" + x + ".*" for x in resultCuisine])))

##### 4. Reading City data from Database one State at a time

In [8]:
curr.execute("SELECT city from address where state='FL'")
resultCity = [item for x in curr.fetchall() for item in x]

##### 5. Inserting records One state at a time and filtering it from cities which are in Database.

In [9]:
df_business_data \
    .filter((df_business_data.state.isin("FL")) & (df_business_data.city.isin(resultCity))) \
    .select("name", "address", "city", "state", "postal_code", "cuisine", "latitude", "longitude", "stars", "review_count") \
    .registerTempTable("staging")

output = spark.sql("select * from staging")


stagedData = [tuple(x) for x in output.collect()]
joinValues = ','.join(['%s'] * len(stagedData))


insertValue = "INSERT INTO restaurantstaging \
    (restaurant_name, street, city, state, postal_code, cuisine_name, latitude, longitude, stars, review_count) VALUES{}".format(joinValues)


curr.execute(insertValue, stagedData)
# curr.execute("select * from restaurantstaging")  --Testing Purpose
# result = curr.fetchmany(2)
# for row in result:
#     print(row)

conn.commit()